In [23]:
import os
import subprocess
import pandas as pd
import shutil
import requests
from Bio import SeqIO, AlignIO
from io import StringIO
import shlex
import sys
import copy

In [24]:
'''
4.12.22
Berk Yalcinkaya

Below are some helper functions for this data pipeline. Skip these.'''

'\n4.12.22\nBerk Yalcinkaya\n\nBelow are some helper functions for this data pipeline. Skip these.'

In [25]:
# use this cell to restart
def restart(path):
    '''delete recently made files. Useful for testing. '''
    try:
        shutil.rmtree(path)
    except:
        print("run analysis first")
#restart(project_dir)

In [26]:
def log(texts:list, newline = True, endline = True):
    '''write text to log_file path, assuming file has been created'''
    f = open(log_file, "a")
    if newline:
        f.write("\n")
        
    for text in texts:
        f.write(text)
        print(text + "\n")
        f.write("\n")
        

In [27]:
def get_subtype_string(subtypes, API = False):
    '''returns a prefix for file headers or apa calls if API'''
    if len(subtypes)==1:
        return subtypes[0]
    header = ""
    for subtype in subtypes:
        header += f"{subtype}-"
    return header 

In [28]:
# Step 0
# Set up
subtypes = ["H1"] # add subtypes here in any order you wish 
                  # ex: [H1,H6,H8], any subtype is acceptable
subtypes.sort()
subtype_string = get_subtype_string(subtypes) # generate an file header to identify this session

In [29]:
# ADD YOUR OWN PATHS HERE
parent_path = "/Users/berk/vaccine/code/vaccine_design" # absolute path to the location of THIS progran
cd_hit_path = "/Users/berk/venvs/vaccine/bin/cd-hit" # change this
muscle_path = "/Users/berk/opt/muscle" # change this

In [30]:
consensus_path =  os.path.join(parent_path, "protein-consensus-sequence")
project_dir = os.path.join(parent_path, f"data/{subtype_string}") # location of session dir
clustered_path = "clustered.fasta" 
invalid_seq_path = "invalid.fasta"
cleaned_path = "seqs.fasta"
aln_path = "aln.fasta"

In [31]:
# create project directories and log file about sequence data
os.makedirs(project_dir, exist_ok=True)

# change current working directory
os.chdir(project_dir)

# create a log file
# write initial text in it
log_file = "log.txt"
log([subtype_string], newline = False)

H1



In [32]:
def count_sequences(records):
    '''takes a Biopython FastaIO records and count sequences'''
    n = 0
    for _ in records:
        n+=1
    return n

In [33]:
def convert_string_to_seq(fasta_string):
    '''converts fasta text to a StringIO object'''
    fasta_io = StringIO(fasta_string) 
    records = SeqIO.parse(fasta_io, "fasta")
    return records

In [34]:
# Step 1 
# Get data from IRD
# API Call using requests
subtype_string_api = get_subtype_string(subtypes, API = True)

base_url = 'https://www.fludb.org/brc/api/sequence'

# must divide search into pre 2008 and post 2008 to avoid overload
# 10,000 sequences max can be quiered at a time
url1 = f"?datatype=protein&completeseq=y&host=human&family=influenza&toyear=2008&flutype=A&protein=HA&subtype={subtype_string_api}&metadata=uniprotAcc,strainName,subtype&output=fasta"
url2 = f"?datatype=protein&completeseq=y&host=human&family=influenza&fromyear=2009&flutype=A&protein=HA&subtype={subtype_string_api}&metadata=uniprotAcc,strainName,subtype&output=fasta"

# make API call
response1 = requests.get(base_url+url1)
response2 = requests.get(base_url+url2)
log(["IRDB API QUERY", response1.url, response2.url])

sequence_string = response1.text + "\n" + response2.text

# first, convert sequences to Biopython records object
sequences = convert_string_to_seq(sequence_string)

# count sequences and write to file
n = count_sequences(sequences)
log([f"{n} sequences were quiered from the IRD"])

IRDB API QUERY

https://www.fludb.org/brc/api/sequence?datatype=protein&completeseq=y&host=human&family=influenza&toyear=2008&flutype=A&protein=HA&subtype=H1&metadata=uniprotAcc,strainName,subtype&output=fasta

https://www.fludb.org/brc/api/sequence?datatype=protein&completeseq=y&host=human&family=influenza&fromyear=2009&flutype=A&protein=HA&subtype=H1&metadata=uniprotAcc,strainName,subtype&output=fasta

23514 sequences were quiered from the IRD



In [35]:
# step 2
# Sequence cleaning
'''
This script can remove sequences that contain ambiguous amino acids or replace ambiguous AA chars with an X. 
Adapted from Biopython Sequence Cleaner tutoral.

URL: https://biopython.org/wiki/Sequence_Cleaner
'''
def clean(sequences):
    '''open output file and continously write valid sequences to output_fasta, 
    invalid sequences are written to invalid_seq_fasta for observation purposes'''
    ambiguous_aa = ["B", "J", "O", "U", "Z"]
    
    def is_valid_sequence(seq):
        '''Helper method. 
        return True if ambiguous_aa chars are in the string, 
        True if string is correctly formatted'''
        if seq[0]!="M": # sequence must start with M
            return False

        for char in ambiguous_aa:
            if char in seq:
                return False
        return True # if the loop completes, no invalid characters were found in inputted sequence

    invalid_sequence_counter=0
    with open (invalid_seq_path, "w+") as output_invalid:
        with open(cleaned_path, "w+") as output:
            # Using the Biopython fasta parse we can read our fasta input
            for sequence in sequences:
                sequence_string = str(sequence.seq).upper()
                #write valid sequences to output
                if is_valid_sequence(sequence_string):
                    output.write(">" + str(sequence.description) + "\n" + str(sequence_string + "\n\n"))
                else:
                    invalid_sequence_counter+=1
                    output_invalid.write(">" + str(sequence.description) + "\n" + str(sequence_string + "\n\n"))
    log([f"{invalid_sequence_counter} invalid sequences were found"])

In [36]:
# perform cleaning using function above
# After iterating through Biopython FASTAIO generator, it's empty
# must restore the sequence record
sequences = convert_string_to_seq(sequence_string)
clean(sequences)

266 invalid sequences were found



In [37]:
def run_shell(command):
    '''runs any given set of arguments on command line'''
    #print(args)
    with open(log_file, 'a') as f:
        process = subprocess.run(command, stdout=f, stderr = f, shell = True)

In [38]:
def cluster(thresh = 0.99):
    '''performs cd-hit clustering'''
    log(["PERFORMING CD-HIT CLUSTERING", "-------------------------------------------------------"])
    terminal_command = f"{cd_hit_path} -i {cleaned_path} -o {clustered_path} -c {thresh}"
    run_shell(terminal_command)

In [39]:
def align():
    '''performs muscle alignment. May take a long time'''
    log(["MUSCLE ALIGNMENT", "-------------------------------------------------------"])
    terminal_command = f"muscle -align {clustered_path} -output {aln_path}"
    try:
        run_shell(terminal_command)
    except:
        print("ERROR: ensure muscle has been downloaded and activated as an executable using chmod")

In [40]:
def get_consensus():
    '''CONSENSUS SEQUENCE AND ENTROPY ANALYSIS - MODIFIED PROTEIN-CONSENSUS-SEQUENCE PROGRAM'''
    '''
    runs protein-consensus-sequence program from https://github.com/msternke/protein-consensus-sequence.git
   
   NOTE: I have included a modified version of this program in the vaccine_design package,
            no need to download any additional software (besides cd-hit and muscle)
   
    
    Outputs
    ---------------------------------------
    1) calculates consensus sequence with gap removal
    2) generate residues frequency
    3) Entropy data at each position
    4) Entropy plots 
    '''
    terminal_command = f"{consensus_path}/consensus.py -i {aln_path} -o consensus.fasta -c 1"
    try:
        run_shell(terminal_command)
    except:
        log(["ensure that you have downloaded protein-consensus-sequence and it has been placedd in working directory"])
    

In [41]:
# cluster sequences
cluster()

# count clusters for log file 
clustered_records = SeqIO.parse(clustered_path, "fasta")
num_clusters = count_sequences(clustered_records)
log([f"{num_clusters} clusters were created"])

PERFORMING CD-HIT CLUSTERING

-------------------------------------------------------

766 clusters were created



In [42]:
# align sequences
# output of muscle is sent to log file
# BUG: for some reason, output is not written to file
align()

MUSCLE ALIGNMENT

-------------------------------------------------------



In [ ]:
# get the consensus sequence
# entropy is calculated 
get_consensus()